In [2]:
import mne
import mne_bids
import numpy as np
import autoreject as ar
import os

In [3]:
bids_root = "/Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS"
output_dir = "/Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/output"
subjects_dir = "/Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/fs_subjects"

In [13]:
for subject in ["ap", "ca", "ha", "ja", "mv", "wc", "zt"]:
    
    epochs_list = []

    for task in ["clean", "noisy"]:

        path = mne_bids.BIDSPath(root=bids_root, subject=subject, session="01", task=task, datatype="ieeg", suffix="ieeg")
        raw = mne_bids.read_raw_bids(path, extra_params=dict(preload=True))

        montage = raw.get_montage()

        # add fiducials to montage
        montage.add_estimated_fiducials(f"{subjects_dir}/{subject}")

        #trans = mne_bids.get_head_mri_trans(bids_path=path, fs_subject=subject,
        #                                    fs_subjects_dir=subjects_dir)

        # now with fiducials assigned, the montage will be properly converted
        # to "head" which is what MNE requires internally (this is the coordinate
        # system with the origin between LPA and RPA whereas MNI has the origin
        # at the posterior commissure)
        raw.set_montage(montage)

        raw.filter(0.15, 200, n_jobs=-1)
        raw.notch_filter(range(60, 200, 60), method="spectrum_fit", filter_length="5s", n_jobs=-1)

        raw_filt = raw.copy().filter(1, None)

        events, event_id = mne.events_from_annotations(raw, event_id={"house/noisy": 11, "house/clean": 1, "face/noisy": 12, "face/clean": 2})

        epochs = mne.Epochs(raw, events, event_id, baseline=None, detrend=0, tmin=-0.2, tmax=0.8, preload=True)

        epochs_filt = mne.Epochs(raw_filt, events, event_id, baseline=None, detrend=0, tmin=-0.2, tmax=0.8, preload=True)

        reject = ar.get_rejection_threshold(epochs_filt, ch_types="ecog", verbose=True)
        print(reject)
        epochs_filt.drop_bad(reject=reject)

        ica = mne.preprocessing.ICA(n_components=0.95, max_iter="auto")
        ica.fit(epochs_filt)
        epochs_clean = ica.apply(epochs.copy())

        reject = ar.get_rejection_threshold(epochs_clean, ch_types="ecog", verbose=True)
        print(reject)
        epochs_clean.drop_bad(reject=reject)

        epochs_clean.apply_baseline((None, None))

        epochs_list.append(epochs_clean)

    epochs = mne.concatenate_epochs(epochs_list, add_offset=100000)

    epochs.save(f"{output_dir}/epoched/{subject}-epo.fif.gz", overwrite=True)

Extracting parameters from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-clean_ieeg.vhdr...
Setting channel info structure...
Reading 0 ... 268399  =      0.000 ...   268.399 secs...
Reading events from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-clean_events.tsv.
Reading channel info from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_task-clean_channels.tsv.
Reading electrode coords from /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/BIDS/sub-ap/ses-01/ieeg/sub-ap_ses-01_space-ACPC_electrodes.tsv.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.15 - 2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband 

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:    1.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:    8.4s finished


Removed notch frequencies (Hz):
     60.00 : 4346 windows
    120.00 : 4346 windows
    180.00 : 4346 windows
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 sec)

Used Annotations descriptions: ['face/clean', 'house/clean']
Not setting metadata
300 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 300 events and 1001 original time points ...
0 bad epochs dropped
Not setting metadata
300 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 300 events and 1001 original t

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:    2.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    4.6s
[Parallel(n_jobs=4)]: Done  41 out of  41 | elapsed:   12.2s finished


Removed notch frequencies (Hz):
     60.00 : 10865 windows
    120.00 : 10865 windows
    180.00 : 10865 windows
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 sec)

Used Annotations descriptions: ['face/noisy', 'house/noisy']
Not setting metadata
630 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 630 events and 1001 original time points ...
0 bad epochs dropped
Not setting metadata
630 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 630 events and 1001 origina

...

Removed notch frequencies (Hz):
     60.00 : 6206 windows
    120.00 : 6206 windows
    180.00 : 6206 windows
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 sec)

Used Annotations descriptions: ['face/clean', 'house/clean']
Not setting metadata
300 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 300 events and 1001 original time points ...
0 bad epochs dropped
Not setting metadata
300 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 300 events and 1001 original t

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done  58 out of  58 | elapsed:    2.6s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    4.7s
[Parallel(n_jobs=4)]: Done  58 out of  58 | elapsed:   17.3s finished


Removed notch frequencies (Hz):
     60.00 : 15486 windows
    120.00 : 15486 windows
    180.00 : 15486 windows
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 sec)

Used Annotations descriptions: ['face/noisy', 'house/noisy']
Not setting metadata
630 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 630 events and 1001 original time points ...
0 bad epochs dropped
Not setting metadata
630 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 630 events and 1001 origina

/var/folders/_r/kcfrqwd143bfg9vx3jtb4l6w0000gs/T/ipykernel_75829/2821719271.py:51: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(epochs_list, add_offset=100000)


Not setting metadata
798 matching events found
Applying baseline correction (mode: mean)
0 bad epochs dropped
Overwriting existing file.


In [11]:
epochs_disk = mne.read_epochs(f"{output_dir}/epoched/{subject}-epo.fif.gz")

Reading /Users/daniel/Documents/Coding_Projects/GitHub/NMA-FacesHouses/data/output/epoched/ap-epo.fif.gz ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
0 bad epochs dropped
Not setting metadata
930 matching events found
No baseline correction applied
0 projection items activated


In [12]:
epochs_disk.event_id

{'face/clean': 2, 'house/clean': 1, 'face/noisy': 12, 'house/noisy': 11}

In [9]:
epochs.event_id

{'face/clean': 2, 'house/clean': 1, 'face/noisy': 12, 'house/noisy': 11}